In [37]:
import pandas as pd
from dataclasses import dataclass, is_dataclass, asdict
from typing import List, Tuple

In [44]:
@dataclass
class BoundingBox:
    x1: int
    y1: int
    x2: int
    y2: int

@dataclass
class Dialogue:
    text: str
    bb: BoundingBox

@dataclass
class Panel:
    # To be added: features, etc.
    dialogues: List[Dialogue]

@dataclass
class Sample:
    context_panels: List[Panel]
    answer_candidates: List[str]
    answer_target: Dialogue = None

In [12]:
df = pd.read_csv("../dataset/COMICS_ocr_file.csv", ',')
df = df.dropna()

/home/smasip/anaconda3/envs/comics/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3251: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [53]:
samples_list: List[Sample] = []
panel = Panel(dialogues=[])
current_comic = 0
current_panel = 0
added_context_panels = 0

for row in df.itertuples():
    text = row.text
    bb = BoundingBox(
        x1=row.x1, y1=row.y1, 
        x2=row.x2, y2=row.y2
    )
    dialogue = Dialogue(text, bb)

    if len(text) < 3:
        continue

    if current_comic != row.comic_no:
        current_comic = row.comic_no
        current_panel = row.panel_no
        panel = Panel(dialogues=[dialogue])
        new_sample = Sample(
                    context_panels=[panel],
                    answer_candidates=[],
                )
        samples_list.append(new_sample)
        added_context_panels = 1

    if current_panel == row.panel_no: 
        panel.dialogues.append(dialogue)

    else:  
        if len(panel.dialogues) > 0:      
            if added_context_panels == 0:
                new_sample = Sample(
                    context_panels=[panel],
                    answer_candidates=[],
                )
                samples_list.append(new_sample)
                added_context_panels += 1
            else:
                samples_list[-1].context_panels.append(panel)

                if added_context_panels >= 2:
                    added_context_panels = 0
                else:
                    added_context_panels += 1

        current_panel = row.panel_no
        panel = Panel(dialogues=[dialogue])

In [54]:
import json


class EnhancedJSONEncoder(json.JSONEncoder):
        def default(self, o):
            if is_dataclass(o):
                return asdict(o)
            return super().default(o)


with open("../datasets/COMICS/text_only.json", "w", encoding="utf-8") as out:
    json.dump(samples_list, out, cls=EnhancedJSONEncoder, indent = 6)